In [1]:
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xlrd
from openpyxl import load_workbook
import os
from datetime import datetime

In [2]:
characteristics_SHOCK = ['rfid', 'subject', 'room', 'cohort', 'trial_id', 'drug', 'box',
       'start_time', 'end_time', 'start_date', 'end_date',
       'total_active_lever_presses', 'total_inactive_lever_presses',
       'total_shocks', 'total_reward', 'rewards_after_first_shock',
       'rewards_got_shock', 'reward_timestamps']

In [3]:
rfid_coc = pd.read_csv('/Users/yunyihuang/Desktop/gl_data/rfid_cocaine.csv', index_col=0)

In [4]:
# get all the sheetnames in one excel workbook
def get_sheetnames_xlsx(file_name):
    wb = load_workbook(file_name, read_only=True, keep_links=False)
    return wb.sheetnames

# clean subject id
def clean_subject_id(sid):
    sid = sid.upper()
    if 'F' in sid:
        char = 'F'
    if 'M' in sid:
        char = 'M'
        
    idx = sid.index(char)
    return sid[idx:].split('.')[0]

# convert column names into correct format
def clean_cols(s):
    if 'U' in s:
        return s.replace('U','Active ')
    elif 'V' in s:
        return s.replace('V','Inactive ')
    elif 'Y' in s:
        return s.replace('Y','Reward ')
    else:
        return s

In [19]:
input_path = '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock'
output_path = '/Users/yunyihuang/Desktop/gl_data/GEORGE/COC_SHOCK'
workbooks = [os.path.join(input_path, i) for i in sorted(os.listdir(input_path)) if ('modified_final' in i)]
workbooks

['/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C01.COC.SHOCK_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C02.COC.SHOCK_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C03.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C04.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C05.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C08.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C09.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C10COCAINESHOCK_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dro

In [6]:
len(workbooks)

10

In [100]:
for wb in workbooks:
    worksheets = sorted(get_sheetnames_xlsx(wb))
    print(len(worksheets))
    for ws in worksheets:
        print(ws)
        transform_old_shock(wb,ws)

3
SHOCK01_9-6-17
SHOCK02_9-11-17
SHOCK03_9-13-17
3
PRESHOCK_11-21-17
SHOCK01_11-27-17
SHOCK02_11-29-17
4
PRESHOCK_3-13-2018
SHOCK01_3-15-2018
SHOCK02_3-20-2018
SHOCK03_3-22-2018
4
PRESHOCK_5-21-2018
SHOCK01_5-23-2018
SHOCK02_5-29-2018
SHOCK03_5-31-2018
4
PRESHOCK_9-4-2018
SHOCK01_9-6-2018
SHOCK02_9-11-2018
SHOCK03_9-13-2018
1
SHOCK03_8-2-2019
2
PRESHOCK_10-29-19
SHOCK03_10-30-2019
2
PRESHOCK_2-5-2020
SHOCK03_2-6-2020
2
PRESHOCK_5-6-2020
SHOCK03_5-7-2020


In [93]:
wb = workbooks[0]
worksheets = sorted(get_sheetnames_xlsx(wb))
ws = worksheets[0]
ws

'SHOCK01_9-6-17'

In [98]:
df_existed = pd.read_csv('/Users/yunyihuang/Desktop/testDS_public_trial_shock.csv')
existed = df_existed['subject'].to_numpy()
existed

array(['F1', 'F10', 'F11', 'F1101', 'F1102', 'F1104', 'F1106', 'F1107',
       'F1108', 'F1109', 'F1110', 'F1112', 'F1113', 'F1114', 'F1115',
       'F1116', 'F1117', 'F1118', 'F1119', 'F1121', 'F1122', 'F1123',
       'F1124', 'F1125', 'F1126', 'F1127', 'F1128', 'F1129', 'F1130',
       'F1135', 'F1137', 'F12', 'F13', 'F1301', 'F1302', 'F1303', 'F1304',
       'F1305', 'F1307', 'F1308', 'F1309', 'F1310', 'F1311', 'F1312',
       'F1313', 'F1314', 'F1315', 'F1316', 'F1317', 'F1319', 'F1320',
       'F1331', 'F1335', 'F14', 'F1401', 'F1403', 'F1404', 'F1406',
       'F1407', 'F1410', 'F1411', 'F1412', 'F1413', 'F1416', 'F1417',
       'F1419', 'F1421', 'F1422', 'F1423', 'F1424', 'F1425', 'F1426',
       'F1428', 'F1429', 'F1431', 'F1432', 'F1433', 'F15', 'F1501',
       'F1503', 'F1504', 'F1505', 'F1506', 'F1507', 'F1510', 'F1511',
       'F1512', 'F1513', 'F1514', 'F1515', 'F1516', 'F1517', 'F1519',
       'F1520', 'F1521', 'F1522', 'F1523', 'F1524', 'F1525', 'F1526',
       'F1527', '

In [95]:
print(wb, ws)

/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C01.COC.SHOCK_modified_final.xlsx SHOCK01_9-6-17


In [96]:
transform_old_shock(wb,ws)

In [99]:
def transform_old_shock(wb,ws):
    filepath = os.path.join(input_path, wb)
    df_raw = pd.read_excel(filepath, sheet_name = ws).T.reset_index()

    # modify the header
    new_header = df_raw.iloc[0]     #grab the first row for the header
    df = df_raw[1:]                 #take the data except the header row
    df.columns = new_header 

    # get rid of 0s
    df.replace(0, np.nan, inplace=True)
    df.dropna(how='all', axis=1, inplace=True)
    df.fillna(0,inplace=True)
    
    # group timestamp columns
    colnames = df.columns.tolist()
    reward_shock_begin = colnames.index('Reward # Got Shock 1')
    reward_col_begin = colnames.index('Reward 1')
    reward_col_end = colnames.index('Rewards After First Shock')
    df['Rewards Got Shock'] = df.iloc[:,reward_shock_begin:reward_col_begin].values.tolist()
    df['Reward Timestamps'] = df.iloc[:,reward_col_begin:reward_col_end].values.tolist()
    df.drop(df.iloc[:, reward_shock_begin:reward_col_end], inplace=True, axis=1)
    
    # add extra info
    trial_id = ws.split('_')[0]
    cohort = int(wb.split('/')[-1][1:3])
    df['room'] = ['N/A'] * len(df)
    df['cohort'] = [cohort] * len(df)
    df['trial_id'] = [trial_id] * len(df)
    df['drug'] = ['cocaine'] * len(df)
    df['end_time'] = [datetime.min.time()] * len(df)
    df['end_date'] = [datetime.min.date()] * len(df)
    
    # reorganize columns
    df.rename(columns=str.lower,inplace=True)
    df.columns = df.columns.str.replace(' ','_')
    dff = pd.merge(df, rfid_coc,  how='left', on = ['subject'])
    dff.fillna({'rfid':-999}, inplace=True)
    dff = dff[characteristics_SHOCK]
    dff['start_time'] = dff['start_time'].apply(lambda x: datetime.strptime(x, "%H:%M:%S").time())
    dff['start_date'] = dff['start_date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y").date())
    dff.sort_values(by='subject', inplace=True)
    dff = dff[~dff['subject'].isin(existed)]

    filename = wb.split('/')[-1][:3] + '_' + ws.split('.')[0]+'_transformed.csv' 
    dff.to_csv(os.path.join(output_path, filename))

In [77]:
filepath = os.path.join(input_path, wb)
df_raw = pd.read_excel(filepath, sheet_name = ws).T.reset_index()

# modify the header
new_header = df_raw.iloc[0]     #grab the first row for the header
df = df_raw[1:]                 #take the data except the header row
df.columns = new_header 

# get rid of 0s
df.replace(0, np.nan, inplace=True)
df.dropna(how='all', axis=1, inplace=True)
df.fillna(0,inplace=True)

df.head()

,Subject,Start Date,Start Time,Box,Total Inactive Lever Presses,Total Reward,Total Active Lever Presses,Total Shocks,Reward # Got Shock 1,Reward # Got Shock 2,...,Reward 44,Reward 45,Reward 46,Reward 47,Reward 48,Reward 49,Reward 50,Reward 51,Reward 52,Rewards After First Shock
1,F1,09/06/2017,08:16:23,1,1.0,29.0,31.0,9.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0
2,F2,09/06/2017,08:16:28,2,0.0,41.0,42.0,13.0,2.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0
3,F3,09/06/2017,08:16:32,3,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,F4,09/06/2017,08:16:35,4,0.0,29.0,29.0,9.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0
5,F5,09/06/2017,08:16:38,5,1.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [78]:
colnames = df.columns.tolist()
reward_shock_begin = colnames.index('Reward # Got Shock 1')
reward_col_begin = colnames.index('Reward 1')
reward_col_end = colnames.index('Rewards After First Shock')
df['Rewards Got Shock'] = df.iloc[:,reward_shock_begin:reward_col_begin].values.tolist()
df['Reward Timestamps'] = df.iloc[:,reward_col_begin:reward_col_end].values.tolist()
df.drop(df.iloc[:, reward_shock_begin:reward_col_end], inplace=True, axis=1)
df

,Subject,Start Date,Start Time,Box,Total Inactive Lever Presses,Total Reward,Total Active Lever Presses,Total Shocks,Rewards After First Shock,Rewards Got Shock,Reward Timestamps
1,F1,09/06/2017,08:16:23,1,1.0,29.0,31.0,9.0,28.0,"[1.0, 2.0, 9.0, 13.0, 16.0, 18.0, 22.0, 23.0, ...","[26.0, 54.0, 92.0, 117.0, 194.0, 276.0, 354.0,..."
2,F2,09/06/2017,08:16:28,2,0.0,41.0,42.0,13.0,39.0,"[2.0, 7.0, 8.0, 11.0, 16.0, 20.0, 22.0, 26.0, ...","[12.0, 45.0, 76.0, 144.0, 179.0, 243.0, 329.0,..."
3,F3,09/06/2017,08:16:32,3,2.0,1.0,1.0,0.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
4,F4,09/06/2017,08:16:35,4,0.0,29.0,29.0,9.0,28.0,"[1.0, 2.0, 8.0, 12.0, 18.0, 19.0, 23.0, 24.0, ...","[3.0, 72.0, 255.0, 372.0, 466.0, 535.0, 698.0,..."
5,F5,09/06/2017,08:16:38,5,1.0,2.0,2.0,0.0,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[637.0, 1848.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,F6,09/06/2017,08:16:43,6,0.0,27.0,27.0,8.0,19.0,"[8.0, 9.0, 10.0, 11.0, 13.0, 18.0, 21.0, 24.0,...","[1.0, 162.0, 214.0, 245.0, 304.0, 339.0, 645.0..."
7,F7,09/06/2017,08:16:46,7,1.0,19.0,22.0,6.0,16.0,"[3.0, 8.0, 10.0, 14.0, 16.0, 18.0, 0.0, 0.0, 0...","[25.0, 57.0, 87.0, 112.0, 222.0, 296.0, 327.0,..."
8,F8,09/06/2017,08:17:02,8,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,M1,09/06/2017,08:17:09,9,1.0,12.0,14.0,4.0,9.0,"[3.0, 6.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[14.0, 56.0, 251.0, 293.0, 664.0, 947.0, 1161...."
10,M2,09/06/2017,08:17:11,10,2.0,25.0,31.0,7.0,21.0,"[4.0, 5.0, 6.0, 12.0, 16.0, 18.0, 22.0, 0.0, 0...","[197.0, 278.0, 478.0, 912.0, 984.0, 1070.0, 11..."


In [79]:
trial_id = ws.split('_')[0]
cohort = int(wb.split('/')[-1][1:3])
df['room'] = ['N/A'] * len(df)
df['cohort'] = [cohort] * len(df)
df['trial_id'] = [trial_id] * len(df)
df['drug'] = ['cocaine'] * len(df)
df['end_time'] = [datetime.min.time()] * len(df)
df['end_date'] = [datetime.min.date()] * len(df)

In [80]:
df.shape

(45, 17)

In [81]:
df.rename(columns=str.lower,inplace=True)
df.columns = df.columns.str.replace(' ','_')
df.head()

,subject,start_date,start_time,box,total_inactive_lever_presses,total_reward,total_active_lever_presses,total_shocks,rewards_after_first_shock,rewards_got_shock,reward_timestamps,room,cohort,trial_id,drug,end_time,end_date
1,F1,09/06/2017,08:16:23,1,1.0,29.0,31.0,9.0,28.0,"[1.0, 2.0, 9.0, 13.0, 16.0, 18.0, 22.0, 23.0, ...","[26.0, 54.0, 92.0, 117.0, 194.0, 276.0, 354.0,...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01
2,F2,09/06/2017,08:16:28,2,0.0,41.0,42.0,13.0,39.0,"[2.0, 7.0, 8.0, 11.0, 16.0, 20.0, 22.0, 26.0, ...","[12.0, 45.0, 76.0, 144.0, 179.0, 243.0, 329.0,...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01
3,F3,09/06/2017,08:16:32,3,2.0,1.0,1.0,0.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01
4,F4,09/06/2017,08:16:35,4,0.0,29.0,29.0,9.0,28.0,"[1.0, 2.0, 8.0, 12.0, 18.0, 19.0, 23.0, 24.0, ...","[3.0, 72.0, 255.0, 372.0, 466.0, 535.0, 698.0,...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01
5,F5,09/06/2017,08:16:38,5,1.0,2.0,2.0,0.0,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[637.0, 1848.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01


In [82]:
dff = pd.merge(df, rfid_coc,  how='left', on = ['subject'])
dff

,subject,start_date,start_time,box,total_inactive_lever_presses,total_reward,total_active_lever_presses,total_shocks,rewards_after_first_shock,rewards_got_shock,reward_timestamps,room,cohort,trial_id,drug,end_time,end_date,rfid
0,F1,09/06/2017,08:16:23,1,1.0,29.0,31.0,9.0,28.0,"[1.0, 2.0, 9.0, 13.0, 16.0, 18.0, 22.0, 23.0, ...","[26.0, 54.0, 92.0, 117.0, 194.0, 276.0, 354.0,...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120124701
1,F2,09/06/2017,08:16:28,2,0.0,41.0,42.0,13.0,39.0,"[2.0, 7.0, 8.0, 11.0, 16.0, 20.0, 22.0, 26.0, ...","[12.0, 45.0, 76.0, 144.0, 179.0, 243.0, 329.0,...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120124703
2,F3,09/06/2017,08:16:32,3,2.0,1.0,1.0,0.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120117342
3,F4,09/06/2017,08:16:35,4,0.0,29.0,29.0,9.0,28.0,"[1.0, 2.0, 8.0, 12.0, 18.0, 19.0, 23.0, 24.0, ...","[3.0, 72.0, 255.0, 372.0, 466.0, 535.0, 698.0,...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120117333
4,F5,09/06/2017,08:16:38,5,1.0,2.0,2.0,0.0,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[637.0, 1848.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120117347
5,F6,09/06/2017,08:16:43,6,0.0,27.0,27.0,8.0,19.0,"[8.0, 9.0, 10.0, 11.0, 13.0, 18.0, 21.0, 24.0,...","[1.0, 162.0, 214.0, 245.0, 304.0, 339.0, 645.0...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120117349
6,F7,09/06/2017,08:16:46,7,1.0,19.0,22.0,6.0,16.0,"[3.0, 8.0, 10.0, 14.0, 16.0, 18.0, 0.0, 0.0, 0...","[25.0, 57.0, 87.0, 112.0, 222.0, 296.0, 327.0,...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120117340
7,F8,09/06/2017,08:17:02,8,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120117351
8,M1,09/06/2017,08:17:09,9,1.0,12.0,14.0,4.0,9.0,"[3.0, 6.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[14.0, 56.0, 251.0, 293.0, 664.0, 947.0, 1161....",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120124705
9,M2,09/06/2017,08:17:11,10,2.0,25.0,31.0,7.0,21.0,"[4.0, 5.0, 6.0, 12.0, 16.0, 18.0, 22.0, 0.0, 0...","[197.0, 278.0, 478.0, 912.0, 984.0, 1070.0, 11...",N/A,1,SHOCK01,cocaine,00:00:00,0001-01-01,933000120117338


In [83]:
dff.fillna({'rfid':-999}, inplace=True)
dff = dff[characteristics_SHOCK]
dff

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000120124701,F1,N/A,1,SHOCK01,cocaine,1,08:16:23,00:00:00,09/06/2017,0001-01-01,31.0,1.0,9.0,29.0,28.0,"[1.0, 2.0, 9.0, 13.0, 16.0, 18.0, 22.0, 23.0, ...","[26.0, 54.0, 92.0, 117.0, 194.0, 276.0, 354.0,..."
1,933000120124703,F2,N/A,1,SHOCK01,cocaine,2,08:16:28,00:00:00,09/06/2017,0001-01-01,42.0,0.0,13.0,41.0,39.0,"[2.0, 7.0, 8.0, 11.0, 16.0, 20.0, 22.0, 26.0, ...","[12.0, 45.0, 76.0, 144.0, 179.0, 243.0, 329.0,..."
2,933000120117342,F3,N/A,1,SHOCK01,cocaine,3,08:16:32,00:00:00,09/06/2017,0001-01-01,1.0,2.0,0.0,1.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,933000120117333,F4,N/A,1,SHOCK01,cocaine,4,08:16:35,00:00:00,09/06/2017,0001-01-01,29.0,0.0,9.0,29.0,28.0,"[1.0, 2.0, 8.0, 12.0, 18.0, 19.0, 23.0, 24.0, ...","[3.0, 72.0, 255.0, 372.0, 466.0, 535.0, 698.0,..."
4,933000120117347,F5,N/A,1,SHOCK01,cocaine,5,08:16:38,00:00:00,09/06/2017,0001-01-01,2.0,1.0,0.0,2.0,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[637.0, 1848.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,933000120117349,F6,N/A,1,SHOCK01,cocaine,6,08:16:43,00:00:00,09/06/2017,0001-01-01,27.0,0.0,8.0,27.0,19.0,"[8.0, 9.0, 10.0, 11.0, 13.0, 18.0, 21.0, 24.0,...","[1.0, 162.0, 214.0, 245.0, 304.0, 339.0, 645.0..."
6,933000120117340,F7,N/A,1,SHOCK01,cocaine,7,08:16:46,00:00:00,09/06/2017,0001-01-01,22.0,1.0,6.0,19.0,16.0,"[3.0, 8.0, 10.0, 14.0, 16.0, 18.0, 0.0, 0.0, 0...","[25.0, 57.0, 87.0, 112.0, 222.0, 296.0, 327.0,..."
7,933000120117351,F8,N/A,1,SHOCK01,cocaine,8,08:17:02,00:00:00,09/06/2017,0001-01-01,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,933000120124705,M1,N/A,1,SHOCK01,cocaine,9,08:17:09,00:00:00,09/06/2017,0001-01-01,14.0,1.0,4.0,12.0,9.0,"[3.0, 6.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[14.0, 56.0, 251.0, 293.0, 664.0, 947.0, 1161...."
9,933000120117338,M2,N/A,1,SHOCK01,cocaine,10,08:17:11,00:00:00,09/06/2017,0001-01-01,31.0,2.0,7.0,25.0,21.0,"[4.0, 5.0, 6.0, 12.0, 16.0, 18.0, 22.0, 0.0, 0...","[197.0, 278.0, 478.0, 912.0, 984.0, 1070.0, 11..."


In [84]:
dff['start_time'] = dff['start_time'].apply(lambda x: datetime.strptime(x, "%H:%M:%S").time())
dff

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000120124701,F1,N/A,1,SHOCK01,cocaine,1,08:16:23,00:00:00,09/06/2017,0001-01-01,31.0,1.0,9.0,29.0,28.0,"[1.0, 2.0, 9.0, 13.0, 16.0, 18.0, 22.0, 23.0, ...","[26.0, 54.0, 92.0, 117.0, 194.0, 276.0, 354.0,..."
1,933000120124703,F2,N/A,1,SHOCK01,cocaine,2,08:16:28,00:00:00,09/06/2017,0001-01-01,42.0,0.0,13.0,41.0,39.0,"[2.0, 7.0, 8.0, 11.0, 16.0, 20.0, 22.0, 26.0, ...","[12.0, 45.0, 76.0, 144.0, 179.0, 243.0, 329.0,..."
2,933000120117342,F3,N/A,1,SHOCK01,cocaine,3,08:16:32,00:00:00,09/06/2017,0001-01-01,1.0,2.0,0.0,1.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,933000120117333,F4,N/A,1,SHOCK01,cocaine,4,08:16:35,00:00:00,09/06/2017,0001-01-01,29.0,0.0,9.0,29.0,28.0,"[1.0, 2.0, 8.0, 12.0, 18.0, 19.0, 23.0, 24.0, ...","[3.0, 72.0, 255.0, 372.0, 466.0, 535.0, 698.0,..."
4,933000120117347,F5,N/A,1,SHOCK01,cocaine,5,08:16:38,00:00:00,09/06/2017,0001-01-01,2.0,1.0,0.0,2.0,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[637.0, 1848.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,933000120117349,F6,N/A,1,SHOCK01,cocaine,6,08:16:43,00:00:00,09/06/2017,0001-01-01,27.0,0.0,8.0,27.0,19.0,"[8.0, 9.0, 10.0, 11.0, 13.0, 18.0, 21.0, 24.0,...","[1.0, 162.0, 214.0, 245.0, 304.0, 339.0, 645.0..."
6,933000120117340,F7,N/A,1,SHOCK01,cocaine,7,08:16:46,00:00:00,09/06/2017,0001-01-01,22.0,1.0,6.0,19.0,16.0,"[3.0, 8.0, 10.0, 14.0, 16.0, 18.0, 0.0, 0.0, 0...","[25.0, 57.0, 87.0, 112.0, 222.0, 296.0, 327.0,..."
7,933000120117351,F8,N/A,1,SHOCK01,cocaine,8,08:17:02,00:00:00,09/06/2017,0001-01-01,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,933000120124705,M1,N/A,1,SHOCK01,cocaine,9,08:17:09,00:00:00,09/06/2017,0001-01-01,14.0,1.0,4.0,12.0,9.0,"[3.0, 6.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[14.0, 56.0, 251.0, 293.0, 664.0, 947.0, 1161...."
9,933000120117338,M2,N/A,1,SHOCK01,cocaine,10,08:17:11,00:00:00,09/06/2017,0001-01-01,31.0,2.0,7.0,25.0,21.0,"[4.0, 5.0, 6.0, 12.0, 16.0, 18.0, 22.0, 0.0, 0...","[197.0, 278.0, 478.0, 912.0, 984.0, 1070.0, 11..."


In [85]:
dff['start_date'] = dff['start_date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y").date())
dff

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000120124701,F1,N/A,1,SHOCK01,cocaine,1,08:16:23,00:00:00,2017-09-06,0001-01-01,31.0,1.0,9.0,29.0,28.0,"[1.0, 2.0, 9.0, 13.0, 16.0, 18.0, 22.0, 23.0, ...","[26.0, 54.0, 92.0, 117.0, 194.0, 276.0, 354.0,..."
1,933000120124703,F2,N/A,1,SHOCK01,cocaine,2,08:16:28,00:00:00,2017-09-06,0001-01-01,42.0,0.0,13.0,41.0,39.0,"[2.0, 7.0, 8.0, 11.0, 16.0, 20.0, 22.0, 26.0, ...","[12.0, 45.0, 76.0, 144.0, 179.0, 243.0, 329.0,..."
2,933000120117342,F3,N/A,1,SHOCK01,cocaine,3,08:16:32,00:00:00,2017-09-06,0001-01-01,1.0,2.0,0.0,1.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,933000120117333,F4,N/A,1,SHOCK01,cocaine,4,08:16:35,00:00:00,2017-09-06,0001-01-01,29.0,0.0,9.0,29.0,28.0,"[1.0, 2.0, 8.0, 12.0, 18.0, 19.0, 23.0, 24.0, ...","[3.0, 72.0, 255.0, 372.0, 466.0, 535.0, 698.0,..."
4,933000120117347,F5,N/A,1,SHOCK01,cocaine,5,08:16:38,00:00:00,2017-09-06,0001-01-01,2.0,1.0,0.0,2.0,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[637.0, 1848.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,933000120117349,F6,N/A,1,SHOCK01,cocaine,6,08:16:43,00:00:00,2017-09-06,0001-01-01,27.0,0.0,8.0,27.0,19.0,"[8.0, 9.0, 10.0, 11.0, 13.0, 18.0, 21.0, 24.0,...","[1.0, 162.0, 214.0, 245.0, 304.0, 339.0, 645.0..."
6,933000120117340,F7,N/A,1,SHOCK01,cocaine,7,08:16:46,00:00:00,2017-09-06,0001-01-01,22.0,1.0,6.0,19.0,16.0,"[3.0, 8.0, 10.0, 14.0, 16.0, 18.0, 0.0, 0.0, 0...","[25.0, 57.0, 87.0, 112.0, 222.0, 296.0, 327.0,..."
7,933000120117351,F8,N/A,1,SHOCK01,cocaine,8,08:17:02,00:00:00,2017-09-06,0001-01-01,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,933000120124705,M1,N/A,1,SHOCK01,cocaine,9,08:17:09,00:00:00,2017-09-06,0001-01-01,14.0,1.0,4.0,12.0,9.0,"[3.0, 6.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[14.0, 56.0, 251.0, 293.0, 664.0, 947.0, 1161...."
9,933000120117338,M2,N/A,1,SHOCK01,cocaine,10,08:17:11,00:00:00,2017-09-06,0001-01-01,31.0,2.0,7.0,25.0,21.0,"[4.0, 5.0, 6.0, 12.0, 16.0, 18.0, 22.0, 0.0, 0...","[197.0, 278.0, 478.0, 912.0, 984.0, 1070.0, 11..."


In [86]:
print(pd.to_datetime('08:16:23').time())

08:16:23


In [87]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 0 to 44
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   rfid                          45 non-null     int64  
 1   subject                       45 non-null     object 
 2   room                          45 non-null     object 
 3   cohort                        45 non-null     int64  
 4   trial_id                      45 non-null     object 
 5   drug                          45 non-null     object 
 6   box                           45 non-null     int64  
 7   start_time                    45 non-null     object 
 8   end_time                      45 non-null     object 
 9   start_date                    45 non-null     object 
 10  end_date                      45 non-null     object 
 11  total_active_lever_presses    45 non-null     float64
 12  total_inactive_lever_presses  45 non-null     float64
 13  total_s

In [35]:
filename = ws.split('.')[0]+'_transformed.csv'
filename   
dff.to_csv(os.path.join(output_path, filename))

In [89]:
dff

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000120124701,F1,N/A,1,SHOCK01,cocaine,1,08:16:23,00:00:00,2017-09-06,0001-01-01,31.0,1.0,9.0,29.0,28.0,"[1.0, 2.0, 9.0, 13.0, 16.0, 18.0, 22.0, 23.0, ...","[26.0, 54.0, 92.0, 117.0, 194.0, 276.0, 354.0,..."
1,933000120124703,F2,N/A,1,SHOCK01,cocaine,2,08:16:28,00:00:00,2017-09-06,0001-01-01,42.0,0.0,13.0,41.0,39.0,"[2.0, 7.0, 8.0, 11.0, 16.0, 20.0, 22.0, 26.0, ...","[12.0, 45.0, 76.0, 144.0, 179.0, 243.0, 329.0,..."
2,933000120117342,F3,N/A,1,SHOCK01,cocaine,3,08:16:32,00:00:00,2017-09-06,0001-01-01,1.0,2.0,0.0,1.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,933000120117333,F4,N/A,1,SHOCK01,cocaine,4,08:16:35,00:00:00,2017-09-06,0001-01-01,29.0,0.0,9.0,29.0,28.0,"[1.0, 2.0, 8.0, 12.0, 18.0, 19.0, 23.0, 24.0, ...","[3.0, 72.0, 255.0, 372.0, 466.0, 535.0, 698.0,..."
4,933000120117347,F5,N/A,1,SHOCK01,cocaine,5,08:16:38,00:00:00,2017-09-06,0001-01-01,2.0,1.0,0.0,2.0,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[637.0, 1848.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,933000120117349,F6,N/A,1,SHOCK01,cocaine,6,08:16:43,00:00:00,2017-09-06,0001-01-01,27.0,0.0,8.0,27.0,19.0,"[8.0, 9.0, 10.0, 11.0, 13.0, 18.0, 21.0, 24.0,...","[1.0, 162.0, 214.0, 245.0, 304.0, 339.0, 645.0..."
6,933000120117340,F7,N/A,1,SHOCK01,cocaine,7,08:16:46,00:00:00,2017-09-06,0001-01-01,22.0,1.0,6.0,19.0,16.0,"[3.0, 8.0, 10.0, 14.0, 16.0, 18.0, 0.0, 0.0, 0...","[25.0, 57.0, 87.0, 112.0, 222.0, 296.0, 327.0,..."
7,933000120117351,F8,N/A,1,SHOCK01,cocaine,8,08:17:02,00:00:00,2017-09-06,0001-01-01,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,933000120124705,M1,N/A,1,SHOCK01,cocaine,9,08:17:09,00:00:00,2017-09-06,0001-01-01,14.0,1.0,4.0,12.0,9.0,"[3.0, 6.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[14.0, 56.0, 251.0, 293.0, 664.0, 947.0, 1161...."
9,933000120117338,M2,N/A,1,SHOCK01,cocaine,10,08:17:11,00:00:00,2017-09-06,0001-01-01,31.0,2.0,7.0,25.0,21.0,"[4.0, 5.0, 6.0, 12.0, 16.0, 18.0, 22.0, 0.0, 0...","[197.0, 278.0, 478.0, 912.0, 984.0, 1070.0, 11..."


In [90]:
dff[~dff['subject'].isin(existed)]

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
16,933000120117341,F9,N/A,1,SHOCK01,cocaine,1,09:43:26,00:00:00,2017-09-06,0001-01-01,25.0,15.0,6.0,22.0,19.0,"[3.0, 5.0, 9.0, 13.0, 14.0, 18.0, 0.0, 0.0, 0....","[14.0, 82.0, 111.0, 191.0, 385.0, 822.0, 860.0..."
17,933000120117334,F10,N/A,1,SHOCK01,cocaine,2,09:43:30,00:00:00,2017-09-06,0001-01-01,63.0,222.0,15.0,52.0,51.0,"[1.0, 2.0, 4.0, 17.0, 18.0, 19.0, 22.0, 24.0, ...","[4.0, 33.0, 63.0, 98.0, 127.0, 149.0, 173.0, 2..."
18,933000120117343,F11,N/A,1,SHOCK01,cocaine,3,09:43:32,00:00:00,2017-09-06,0001-01-01,32.0,1.0,7.0,22.0,21.0,"[1.0, 2.0, 4.0, 13.0, 15.0, 17.0, 22.0, 0.0, 0...","[35.0, 56.0, 86.0, 139.0, 306.0, 459.0, 574.0,..."
19,933000120117355,F12,N/A,1,SHOCK01,cocaine,4,09:43:35,00:00:00,2017-09-06,0001-01-01,31.0,0.0,9.0,30.0,29.0,"[1.0, 4.0, 7.0, 13.0, 16.0, 20.0, 27.0, 29.0, ...","[15.0, 40.0, 65.0, 90.0, 116.0, 280.0, 304.0, ..."
20,933000120117346,F13,N/A,1,SHOCK01,cocaine,5,09:43:37,00:00:00,2017-09-06,0001-01-01,26.0,0.0,6.0,21.0,19.0,"[2.0, 3.0, 6.0, 14.0, 18.0, 19.0, 0.0, 0.0, 0....","[18.0, 45.0, 207.0, 471.0, 591.0, 724.0, 1029...."
21,933000120124699,F14,N/A,1,SHOCK01,cocaine,6,09:43:40,00:00:00,2017-09-06,0001-01-01,68.0,1.0,12.0,40.0,39.0,"[1.0, 5.0, 6.0, 12.0, 13.0, 16.0, 22.0, 24.0, ...","[46.0, 80.0, 144.0, 174.0, 200.0, 225.0, 247.0..."
22,933000120124700,F15,N/A,1,SHOCK01,cocaine,7,09:43:45,00:00:00,2017-09-06,0001-01-01,10.0,0.0,2.0,6.0,3.0,"[3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[103.0, 263.0, 370.0, 411.0, 805.0, 1162.0, 0...."
23,933000120124697,F16,N/A,1,SHOCK01,cocaine,8,09:43:50,00:00:00,2017-09-06,0001-01-01,47.0,1.0,10.0,34.0,30.0,"[4.0, 5.0, 7.0, 15.0, 16.0, 19.0, 21.0, 22.0, ...","[125.0, 155.0, 193.0, 235.0, 261.0, 285.0, 341..."
24,933000120117339,M27,N/A,1,SHOCK01,cocaine,9,09:43:54,00:00:00,2017-09-06,0001-01-01,3.0,2.0,1.0,2.0,0.0,"[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[13.0, 42.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
25,933000120117350,M28,N/A,1,SHOCK01,cocaine,10,09:43:57,00:00:00,2017-09-06,0001-01-01,19.0,1.0,6.0,17.0,14.0,"[3.0, 7.0, 10.0, 11.0, 13.0, 14.0, 0.0, 0.0, 0...","[82.0, 335.0, 470.0, 560.0, 712.0, 1110.0, 140..."
